# Dynamical Unit

Dynamical Unit 是 BrainCell 框架的顶层抽象，通过 **DiffEqState** 和 **DiffEqModule** 实现神经元动态的数学描述。该层将生物学过程转化为严格的微分方程系统，是连接微观生物机制与宏观系统行为的桥梁。


## 一、DiffEqState：神经动态系统的状态变量管理

### 1.1 数学背景
神经动态系统本质上是**高维非线性动力系统**，其状态由一组变量描述：
- **膜电位 $V$**：神经元的核心状态变量，决定神经元的兴奋性
- **离子通道门控变量**：描述通道开放概率
- **离子浓度**：影响反转电位和离子驱动力
- **突触变量**：介导神经元间的信号传递

这些变量构成系统的**状态空间**，其演化遵循微分方程：
$$ \frac{d\mathbf{x}}{dt} = \mathbf{f}(\mathbf{x}, t, \mathbf{u}) $$
其中，$\mathbf{x}$ 是状态向量，$\mathbf{u}$ 是外部输入，$\mathbf{f}$ 是状态演化函数。


### 1.2 生物学应用
在神经科学中，DiffEqState 直接对应生物实体的**可测量状态**：
- **膜电位**：反映神经元的兴奋状态
- **门控变量**：刻画离子通道的开放/关闭概率
- **离子浓度**：调控神经元的长期兴奋性




## 二、DiffEqModule：神经动态系统的演化规则

### 2.1 数学背景
DiffEqModule 定义状态变量的**演化规则**，即微分方程的具体形式。对于神经动态系统，常见的方程类型包括：

#### 2.1.1 膜电位方程
基于电流守恒定律：
$$ C_m \frac{dV}{dt} = I_{\text{input}} - \sum_{i} I_i $$
其中，$I_i$ 是第 $i$ 种离子通道的电流（如 $I_{\text{Na}}$、$I_{\text{K}}$）。

#### 2.1.2 门控变量方程
基于 Markov 过程的双态模型：
$$ \frac{dx}{dt} = \alpha_x(V)(1-x) - \beta_x(V)x $$
其中，$\alpha_x$ 和 $\beta_x$ 是依赖于膜电位的速率常数（如 HH 模型中的 $m$、$h$、$n$）。

#### 2.1.3 离子浓度方程
描述离子跨膜流动引起的浓度变化：
$$ \frac{d[X]_{\text{in}}}{dt} = \frac{I_X}{zXF} \cdot \frac{A}{V_{\text{cell}}} - \text{泵/交换机制} $$
其中，$z$ 是离子价态，$F$ 是法拉第常数，$A$ 是膜面积，$V_{\text{cell}}$ 是细胞体积。


### 2.2 生物学应用
DiffEqModule 将生物物理机制转化为数学方程，典型应用包括：

#### 2.2.1 HH 模型的电流计算


#### 2.2.2 突触可塑性模型
描述长时程增强和长时程抑制的机制：
$$ \frac{dW}{dt} = \eta \cdot f(V_{\text{pre}}, V_{\text{post}}, [\text{Ca}^{2+}]_{\text{post}}) $$
其中，$W$ 是突触权重，$eta$ 是学习率，$f$ 是依赖于突触前/后神经元活动和钙离子浓度的可塑性函数。


## 三、两者关系：状态与演化的协同建模

### 3.1 数学抽象
- **DiffEqState**：定义动力系统的**状态空间**
- **DiffEqModule**：定义状态变量的**演化规则**

两者共同构成完整的**初值问题**：
$$ \frac{d\mathbf{x}}{dt} = \mathbf{f}(\mathbf{x}, t) \quad \text{with} \quad \mathbf{x}(0) = \mathbf{x}_0 $$
其中，$\mathbf{x}_0$ 由 DiffEqState 提供，$\mathbf{f}$ 由 DiffEqModule 实现。


### 3.2 生物学映射
- **状态变量**（DiffEqState）对应生物系统的**可测量状态**
- **演化规则**（DiffEqModule）对应生物系统的**物理/化学规律**

例如，在模拟神经元放电时：
- DiffEqState 存储当前的膜电位和门控变量值
- DiffEqModule 根据 HH 方程计算膜电位和门控变量的导数
- 数值积分器根据导数更新状态变量，形成闭环


### 3.3 计算框架中的协作
BrainCell 通过 JAX 实现高效的自动微分和向量化计算，使动态单元具备：
- **可组合性**：不同的 DiffEqState 和 DiffEqModule 可自由组合，构建从单细胞到网络的多层次模型
- **可扩展性**：通过继承基类，用户可自定义新的状态变量和演化规则
- **高性能**：JAX 的即时编译（JIT）和向量化（vmap）优化大规模神经模拟

例如，通过组合多个 DiffEqModule 可实现复杂网络：


## 四、神经科学中的典型应用

### 4.1 动作电位的产生与传播
通过 HH 模型的 DiffEqModule，复现动作电位的全周期：
1. 膜电位去极化达到阈值，触发 Na⁺ 通道快速激活（m 门上升）
2. Na⁺ 内流导致膜电位迅速上升（动作电位上升相）
3. Na⁺ 通道失活（h 门下降）+ K⁺ 通道激活（n 门上升），导致复极化
4. K⁺ 外流持续，形成超极化
5. 门控变量恢复初始值，膜电位回到静息水平


### 4.2 突触可塑性的计算模型
通过扩展 DiffEqModule，实现依赖于活动的突触权重更新规则（如 STDP）：
$$ \Delta W = \begin{cases}
A_+ \exp\left(-\frac{\Delta t}{\tau_+}\right) & \text{if } \Delta t > 0 \\
-A_- \exp\left(\frac{\Delta t}{\tau_-}\right) & \text{if } \Delta t < 0
\end{cases} $$
其中，$\Delta t = t_{\text{post}} - t_{\text{pre}}$是突触后与突触前 spike 的时间差。

这种数学建模方法使我们能够从分子机制（如离子通道动力学）出发，定量预测神经元和神经网络的宏观行为，架起微观生物机制与宏观神经功能之间的桥梁。